<a href="https://colab.research.google.com/github/FadhilahAyuLansari/4212211036_AAS_Machine_Vision/blob/main/4212211036_aas_machine_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
 import torch
 import numpy as np
 from torchvision import transforms, models
 from torch.utils.data import Dataset, DataLoader
 from torch import nn, optim
 from PIL import Image
 from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score
 from tqdm import tqdm

In [ ]:
train_data = pd.read_csv('/emnist-bymerge-train.csv', header=None, nrows=500)
 test_data = pd.read_csv('/emnist-bymerge-test.csv', header=None, nrows=500)

In [ ]:
def preprocess_image(data):
    data = np.clip(data, 0, 255).astype(np.uint8).reshape(28, 28)
    return Image.fromarray(data).convert("RGB")

In [ ]:
class CustomDataset(Dataset):
    def _init_(self, dataframe, transform=None):
        """
        Initialize the dataset with a DataFrame and optional image transformations.
        """
        self.dataframe = dataframe
        self.transform = transform
    def _len_(self):
        """
        Returns the total number of samples in the dataset.
        """
        return len(self.dataframe)
    def _getitem_(self, idx):
        """
        Retrieves the image and label for a given index, applies preprocessing and transformations.
        """
        label = self.dataframe.iloc[idx, 0]
        img_data = self.dataframe.iloc[idx, 1:].values
        image = preprocess_image(img_data)
        if self.transform:
            image = self.transform(image)
        return image, label

    transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
    ])

In [ ]:
# Create datasets and data loaders for training and validation
 # Use train_data and test_data instead of data_train and data_val
 train_dataset = CustomDataset(train_data, transform=CustomDataset.transform)  # Use CustomDataset.transform
 val_dataset = CustomDataset(test_data, transform=CustomDataset.transform)  # Use CustomDataset.transform
 train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)  # Training data loader
 val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)    # Validation data loader

In [ ]:
# Initialize a pretrained AlexNet model for transfer learning
model = models.alexnet(pretrained=True)
model.classifier[6] = nn.Linear(4096, 200)

In [ ]:
# Freeze feature extraction layers to only train the classifier
for param in model.features.parameters():
    param.requires_grad = False

In [ ]:
# Define loss function and optimizer
 criterion = nn.CrossEntropyLoss()
 optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# Configure the device for GPU acceleration (if available)
 device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 model.to(device)

In [ ]:
# Training Loop
 from sklearn.model_selection import LeaveOneOut
 from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score

In [ ]:
# Convert the training data into a NumPy array for LOOCV
 data_array = train_data.to_numpy()

In [ ]:
# Initialize lists to store predictions and labels for evaluation
 all_preds, all_labels = [], []
 print("Starting LOOCV...")

In [ ]:
# Leave-One-Out Cross Validation implementation
loo = LeaveOneOut()
for train_idx, test_idx in tqdm(loo.split(data_array)):
    # Split data into training and test sets for this fold
    train_samples = data_array[train_idx]
    test_sample = data_array[test_idx]
    # Create datasets and dataloaders for the current LOOCV split
    train_dataset = CustomDataset(pd.DataFrame(train_samples), transform=CustomDataset.transform)
    test_dataset = CustomDataset(pd.DataFrame(test_sample), transform=CustomDataset.transform)
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
     # Reinitialize the model and optimizer for each LOOCV iteration
    model = models.alexnet(pretrained=True)
    model.classifier[6] = nn.Linear(4096, 200)
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    # Training loop for the current fold
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    # Validation loop for the current fold
    model.eval()
    with torch.no_grad(): # Corrected indentation here
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            all_preds.append(torch.argmax(outputs, dim=1).cpu().item())
            all_labels.append(labels.cpu().item())

In [ ]:
# Calculate evaluation metrics
 conf_matrix = confusion_matrix(all_labels, all_preds)  # Confusion matrix
 accuracy = accuracy_score(all_labels, all_preds)      # Accuracy score
 precision = precision_score(all_labels, all_preds, average="macro", zero_division=0)  # Precision score
 f1 = f1_score(all_labels, all_preds, average="macro") # F1 score

In [ ]:
# Display evaluation results
print("\nEvaluation Results:")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1-Score: {f1:.4f}")